<a href="https://colab.research.google.com/github/kpomazi/2021-Better-Working-World-Data-Challenge/blob/main/4_match_lightning_with_data_glm_asos_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Utils

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
dataset_name = 'glm_2020'
pth_input = f"/content/drive/MyDrive/FNN/dangerous_lightnings/1_data_download/output/lightning/{dataset_name}.csv"
pth_output = "/content/drive/MyDrive/FNN/dangerous_lightnings/4_match_lightning_with_data/output/temp"

In [3]:
!pip install -q geopandas
!pip install -q haversine
!pip install -q pygeos
!pip install -q rasterio
!pip install -q rioxarray

     |████████████████████████████████| 1.1 MB 8.1 MB/s 
     |████████████████████████████████| 7.8 MB 51.3 MB/s 
     |████████████████████████████████| 16.6 MB 37.6 MB/s 
     |████████████████████████████████| 2.2 MB 7.0 MB/s 
     |████████████████████████████████| 20.9 MB 1.4 MB/s 
     |████████████████████████████████| 53 kB 1.4 MB/s 


In [4]:
import os
import io
import urllib.request
import pandas as pd
import numpy as np
import geopandas as gpd
from haversine import haversine
from datetime import datetime as dt, timedelta
import time
from tqdm.notebook import trange, tqdm
import xarray as xr
import rasterio
import rioxarray

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
<ipython-input-4-61f902f1c2b0>:6: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  impo

In [5]:
dflg = pd.read_csv(pth_input, index_col=0, delimiter=';')
print(dflg.shape)
dflg.head()

(5978004, 8)


,flash_id,flash_time_offset_of_first_event,flash_time_offset_of_last_event,flash_lat,flash_lon,flash_area,flash_energy,flash_quality_flag
",",,,,,,,,
"0,0",7562,2020-09-01 15:56:53.642332077,2020-09-01 15:56:53.849090576,29.399868,-82.883220,1.252098e+09,3.942836e-13,0.0
"1,1",6668,2020-09-01 15:54:37.409402847,2020-09-01 15:54:37.510875701,29.364891,-82.836250,2.083015e+08,1.972844e-13,0.0
"2,2",61143,2020-09-01 15:21:55.164800643,2020-09-01 15:21:55.196462631,29.748596,-82.283790,6.973905e+07,1.128511e-14,0.0
"3,3",61135,2020-09-01 15:21:53.876558303,2020-09-01 15:21:53.882661819,27.976433,-82.647300,4.765756e+08,7.328486e-14,0.0
"4,4",57722,2020-09-01 15:10:51.160070419,2020-09-01 15:10:51.496149063,28.976896,-82.880585,2.765146e+08,1.382846e-13,0.0


In [6]:
dflg = dflg[['flash_time_offset_of_first_event','flash_lat','flash_lon','flash_area','flash_energy','flash_quality_flag']]
dflg.rename({'flash_time_offset_of_first_event':'dttime', 'flash_lat':'lat', 'flash_lon':'lon'}, axis=1, inplace=True)
dflg.reset_index(inplace=True, drop=True)
dflg.head()

,dttime,lat,lon,flash_area,flash_energy,flash_quality_flag
0,2020-09-01 15:56:53.642332077,29.399868,-82.883220,1.252098e+09,3.942836e-13,0.0
1,2020-09-01 15:54:37.409402847,29.364891,-82.836250,2.083015e+08,1.972844e-13,0.0
2,2020-09-01 15:21:55.164800643,29.748596,-82.283790,6.973905e+07,1.128511e-14,0.0
3,2020-09-01 15:21:53.876558303,27.976433,-82.647300,4.765756e+08,7.328486e-14,0.0
4,2020-09-01 15:10:51.160070419,28.976896,-82.880585,2.765146e+08,1.382846e-13,0.0


In [7]:
if 'time' in dflg.columns:
  dflg.rename({'time':'dttime'}, axis=1, inplace=True)
if 'latitude' in dflg.columns:
  dflg.rename({'latitude':'lat'}, axis=1, inplace=True)
if 'longitude' in dflg.columns:
  dflg.rename({'longitude':'lon'}, axis=1, inplace=True)

In [8]:
dflg.lat.nunique()

2773356

In [9]:
dflg['lat_round'] = dflg.lat.round(2)
dflg.lat_round.nunique()

649

In [10]:
dflg['lon_round'] = dflg.lon.round(2)
dflg.lon_round.nunique()

761

In [11]:
dflg.head()

,dttime,lat,lon,flash_area,flash_energy,flash_quality_flag,lat_round,lon_round
0,2020-09-01 15:56:53.642332077,29.399868,-82.883220,1.252098e+09,3.942836e-13,0.0,29.40,-82.88
1,2020-09-01 15:54:37.409402847,29.364891,-82.836250,2.083015e+08,1.972844e-13,0.0,29.36,-82.84
2,2020-09-01 15:21:55.164800643,29.748596,-82.283790,6.973905e+07,1.128511e-14,0.0,29.75,-82.28
3,2020-09-01 15:21:53.876558303,27.976433,-82.647300,4.765756e+08,7.328486e-14,0.0,27.98,-82.65
4,2020-09-01 15:10:51.160070419,28.976896,-82.880585,2.765146e+08,1.382846e-13,0.0,28.98,-82.88


In [12]:
dflg['latlon'] = dflg['lat'].map(str) + dflg['lon'].map(str)
dflg['latlon'].nunique()

5977978

In [13]:
dflg['latlon_round'] = dflg['lat_round'].map(str) + dflg['lon_round'].map(str)
dflg['latlon_round'].nunique()

486366

# Weather parquet

Todo:  
Weather df: minden kiszámított átlag legyen benne, abból olvassuk ki. Átlagolás aztán join.

In [14]:
wsttdata = "/content/drive/MyDrive/FNN/dangerous_lightnings/1_data_download/weather_station.csv"
dfstt = pd.read_csv(wsttdata)

In [15]:
weather_dict = {}
for stid in tqdm(dfstt.stid):
  #CHANGE TO YOUR FILE LOCATION
  wstt_path = f'/content/drive/MyDrive/FNN/dangerous_lightnings/1_data_download/output/weather/{stid}.csv'
  data = pd.read_csv(wstt_path,index_col=0)
  data[['tmpc','relh', 'sknt', 'p01i']] = data[['tmpc','relh', 'sknt', 'p01i']].apply(pd.to_numeric, errors='coerce')
  data = data.dropna()
  data['datetime'] = pd.to_datetime(data['valid'])
  weather_dict[stid]=data

  0%|          | 0/104 [00:00<?, ?it/s]

In [16]:
weather_dict.keys()

dict_keys(['AAF', 'BOW', 'BCT', 'BKV', 'FIN', 'XMR', 'VSH', 'NFJ', 'CLW', '2IS', 'CEW', 'CTY', 'CGC', 'DAB', '54J', 'DED', 'DTS', 'EGI', 'VPS', 'FHB', 'FXE', 'FMY', 'FLL', 'RSW', 'FPR', 'GNV', 'HWO', 'HST', 'HRT', 'IMM', 'X40', 'INF', 'CRG', 'JAX', 'NIP', 'VQQ', 'NEN', '42J', 'EYW', 'NQX', 'ISM', 'LCQ', 'LAL', 'X07', 'LNA', 'LEE', '24J', 'MCF', 'MTH', 'MKY', 'MAI', 'NRB', 'MLB', 'MIA', 'TMB', 'OPF', 'NDZ', 'APF', 'TTS', 'EVB', 'OCF', 'OBE', 'ORL', 'MCO', 'OMN', '28J', 'XFL', 'ECP', 'PFN', 'TDR', 'COF', 'NPA', 'PNS', 'FPY', '40J', 'PCM', 'PMP', 'PGD', '2J9', 'PIE', 'SPG', 'SFB', 'SRQ', 'SEF', 'SGJ', 'TLH', 'TPA', 'TPF', 'VVG', 'TIX', '1J0', 'BCR', 'PAM', 'X26', 'VDF', 'VNC', 'VRB', 'F45', 'PBI', 'NSE', 'X60', 'GIF', 'SUA', 'ZPH'])

In [17]:
weather_dict['OCF'].head()

,station,valid,lon,lat,tmpc,relh,sknt,p01i,datetime
11,OCF,2021-12-01 00:51,-82.221,29.1636,9.39,74.05,0.0,0.0,2021-12-01 00:51:00
24,OCF,2021-12-01 01:51,-82.221,29.1636,7.78,79.13,0.0,0.0,2021-12-01 01:51:00
37,OCF,2021-12-01 02:51,-82.221,29.1636,7.22,79.36,3.0,0.0,2021-12-01 02:51:00
50,OCF,2021-12-01 03:51,-82.221,29.1636,6.72,85.06,0.0,0.0,2021-12-01 03:51:00
61,OCF,2021-12-01 04:51,-82.221,29.1636,6.72,85.06,0.0,0.0,2021-12-01 04:51:00


In [28]:
weather_dict['OCF'].tail()

,station,valid,lon,lat,tmpc,relh,sknt,p01i,datetime
121482,OCF,2022-12-21 06:51,-82.221,29.1636,13.28,96.78,4.0,0.00,2022-12-21 06:51:00
121485,OCF,2022-12-21 07:07,-82.221,29.1636,13.00,100.00,7.0,0.01,2022-12-21 07:07:00
121492,OCF,2022-12-21 07:37,-82.221,29.1636,13.00,100.00,8.0,0.03,2022-12-21 07:37:00
121494,OCF,2022-12-21 07:44,-82.221,29.1636,13.00,100.00,5.0,0.03,2022-12-21 07:44:00
121497,OCF,2022-12-21 07:51,-82.221,29.1636,13.28,96.78,6.0,0.04,2022-12-21 07:51:00


In [18]:
cls = ['station', 'datetime', 'lon', 'lat',  
    'tmpc', 
    'tmpc-1h', 'tmpc-2h','tmpc-3h', 'tmpc-4h', 
    'relh', 
    'relh-1h', 'relh-2h', 'relh-3h', 'relh-4h', 
    'sknt',
    'sknt-1h', 'sknt-2h', 'sknt-3h', 'sknt-4h', 
    'p01i', 
    'p01i-1h', 'p01i-2h', 'p01i-3h', 'p01i-4h']

weather_dict2 = {}

for key, value in tqdm(weather_dict.items()):
  dfwstt = pd.DataFrame(columns=cls)  
  knm = ['tmpc', 'relh', 'sknt', 'p01i']
  lst=[]
  for index, row in tqdm(value.iterrows(), total=value.shape[0]):
    dct = {}
    dct['station'] = row.station
    dct['datetime'] = row.datetime
    dct['lon'] = row.lon
    dct['lat'] = row.lat

    for i in [0, 1, 2, 3, 4]:
        dtstart = row.datetime
        dfsmp = value[(value.datetime >= dtstart - timedelta(hours=i)) & (value.datetime < dtstart - timedelta(hours=i-1)) &(value.datetime <= dtstart)]
        if i == 0:
          for k in knm:
            nm = f'{k}'
            dct[nm] = round(dfsmp[k].mean(),2)
        else:
          for k in knm:
                  nm = f'{k}-{i}h'
                  dct[nm] = round(dfsmp[k].mean(),2)
    lst.append(dct)
  dfwstt = dfwstt.append(pd.DataFrame(lst))
  dfwstt['dttime_hour'] = pd.to_datetime(dfwstt.datetime).round('1h')
  dfwstt2 = dfwstt.set_index(['dttime_hour'])
  print(dfwstt.shape[0]-dfwstt2.shape[0])
  weather_dict2[key] = dfwstt2.dropna()

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/11426 [00:00<?, ?it/s]

0


  0%|          | 0/17734 [00:00<?, ?it/s]

0


...

0


  0%|          | 0/12240 [00:00<?, ?it/s]

0


  0%|          | 0/18341 [00:00<?, ?it/s]

0


In [19]:
weather_dict2.keys()

dict_keys(['AAF', 'BOW', 'BCT', 'BKV', 'FIN', 'XMR', 'VSH', 'NFJ', 'CLW', '2IS', 'CEW', 'CTY', 'CGC', 'DAB', '54J', 'DED', 'DTS', 'EGI', 'VPS', 'FHB', 'FXE', 'FMY', 'FLL', 'RSW', 'FPR', 'GNV', 'HWO', 'HST', 'HRT', 'IMM', 'X40', 'INF', 'CRG', 'JAX', 'NIP', 'VQQ', 'NEN', '42J', 'EYW', 'NQX', 'ISM', 'LCQ', 'LAL', 'X07', 'LNA', 'LEE', '24J', 'MCF', 'MTH', 'MKY', 'MAI', 'NRB', 'MLB', 'MIA', 'TMB', 'OPF', 'NDZ', 'APF', 'TTS', 'EVB', 'OCF', 'OBE', 'ORL', 'MCO', 'OMN', '28J', 'XFL', 'ECP', 'PFN', 'TDR', 'COF', 'NPA', 'PNS', 'FPY', '40J', 'PCM', 'PMP', 'PGD', '2J9', 'PIE', 'SPG', 'SFB', 'SRQ', 'SEF', 'SGJ', 'TLH', 'TPA', 'TPF', 'VVG', 'TIX', '1J0', 'BCR', 'PAM', 'X26', 'VDF', 'VNC', 'VRB', 'F45', 'PBI', 'NSE', 'X60', 'GIF', 'SUA', 'ZPH'])

In [20]:
weather_dict2['AAF'].head()

,station,datetime,lon,lat,tmpc,tmpc-1h,tmpc-2h,tmpc-3h,tmpc-4h,relh,...,sknt,sknt-1h,sknt-2h,sknt-3h,sknt-4h,p01i,p01i-1h,p01i-2h,p01i-3h,p01i-4h
dttime_hour,,,,,,,,,,,,,,,,,,,,,
2021-12-01 05:00:00,AAF,2021-12-01 04:53:00,-85.0274,29.7276,5.00,5.61,5.61,7.22,7.78,85.54,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-12-01 06:00:00,AAF,2021-12-01 05:53:00,-85.0274,29.7276,5.00,5.00,5.61,5.61,7.22,82.22,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-12-01 07:00:00,AAF,2021-12-01 06:53:00,-85.0274,29.7276,3.89,5.00,5.00,5.61,5.61,88.87,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-12-01 08:00:00,AAF,2021-12-01 07:53:00,-85.0274,29.7276,3.89,3.89,5.00,5.00,5.61,85.76,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-12-01 09:00:00,AAF,2021-12-01 08:53:00,-85.0274,29.7276,3.89,3.89,3.89,5.00,5.00,88.87,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
dfwstt.head()

,station,datetime,lon,lat,tmpc,tmpc-1h,tmpc-2h,tmpc-3h,tmpc-4h,relh,...,sknt-1h,sknt-2h,sknt-3h,sknt-4h,p01i,p01i-1h,p01i-2h,p01i-3h,p01i-4h,dttime_hour
0,ZPH,2021-12-01 00:55:00,-82.1559,28.2281,11.22,NaN,NaN,NaN,NaN,72.13,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2021-12-01 01:00:00
1,ZPH,2021-12-01 01:15:00,-82.1559,28.2281,9.89,11.22,NaN,NaN,NaN,77.93,...,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,2021-12-01 01:00:00
2,ZPH,2021-12-01 01:35:00,-82.1559,28.2281,8.89,10.56,NaN,NaN,NaN,84.32,...,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,2021-12-01 02:00:00
3,ZPH,2021-12-01 01:55:00,-82.1559,28.2281,8.89,10.00,NaN,NaN,NaN,82.72,...,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,2021-12-01 02:00:00
4,ZPH,2021-12-01 02:15:00,-82.1559,28.2281,8.61,9.22,11.22,NaN,NaN,85.92,...,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,2021-12-01 02:00:00


In [22]:
import pickle
with open(f"{pth_output}/weather_dict.pkl", 'wb') as f:
    pickle.dump(weather_dict2, f)

In [23]:
import pickle
with open(f"{pth_output}/weather_dict.pkl", 'rb') as f:
    weather_dict2 = pickle.load(f)

In [24]:
dfwstt.shape

(18341, 25)

In [25]:
dfwstt2 = dfwstt.dropna()
dfwstt2.shape

(17611, 25)

In [26]:
dfwstt.to_csv(f"{pth_output}/dfwstt_2022.csv")

In [27]:
dfwstt2.to_csv(f"{pth_output}/dfwstt2_2022.csv")

In [29]:
df_weather = pd.DataFrame()
for key, value in weather_dict2.items():
    value['station'] = key
    df_weather = df_weather.append(value)

In [30]:
df_weather.shape

(1344733, 24)

In [ ]:
df_weather.to_csv(f"{pth_output}/df_weather_2022.csv")

# ASOS

In [ ]:
dfwstt2 = pd.read_csv(f"{pth_output}/dfwstt2.csv",index_col=0)
dfwstt2.head()

,station,datetime,lon,lat,tmpc,tmpc-1h,tmpc-2h,tmpc-3h,tmpc-4h,relh,...,sknt,sknt-1h,sknt-2h,sknt-3h,sknt-4h,p01i,p01i-1h,p01i-2h,p01i-3h,p01i-4h
4,AAF,2020-01-01 04:53:00,-85.0274,29.7276,6.72,6.72,7.22,8.89,8.28,78.66,...,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0
5,AAF,2020-01-01 05:53:00,-85.0274,29.7276,5.61,6.72,6.72,7.22,8.89,81.98,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
6,AAF,2020-01-01 06:53:00,-85.0274,29.7276,5.61,5.61,6.72,6.72,7.22,81.98,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,AAF,2020-01-01 07:53:00,-85.0274,29.7276,5.61,5.61,5.61,6.72,6.72,81.98,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,AAF,2020-01-01 08:53:00,-85.0274,29.7276,6.11,5.61,5.61,5.61,6.72,79.19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
dfwstt2.shape

(1260858, 24)

In [ ]:
dfwstt2['dttime_hour'] = pd.to_datetime(dfwstt2.datetime).round('1h')
dfwstt2 = dfwstt2.set_index(['station', 'dttime_hour'])
dfwstt2.head()

datetime      lon      lat  tmpc  \
station dttime_hour                                                        
AAF     2020-01-01 05:00:00  2020-01-01 04:53:00 -85.0274  29.7276  6.72   
        2020-01-01 06:00:00  2020-01-01 05:53:00 -85.0274  29.7276  5.61   
        2020-01-01 07:00:00  2020-01-01 06:53:00 -85.0274  29.7276  5.61   
        2020-01-01 08:00:00  2020-01-01 07:53:00 -85.0274  29.7276  5.61   
        2020-01-01 09:00:00  2020-01-01 08:53:00 -85.0274  29.7276  6.11   

                             tmpc-1h  tmpc-2h  tmpc-3h  tmpc-4h   relh  \
station dttime_hour                                                      
AAF     2020-01-01 05:00:00     6.72     7.22     8.89     8.28  78.66   
        2020-01-01 06:00:00     6.72     6.72     7.22     8.89  81.98   
        2020-01-01 07:00:00     5.61     6.72     6.72     7.22  81.98   
        2020-01-01 08:00:00     5.61     5.61     6.72     6.72  81.98   
        2020-01-01 09:00:00     5.61     5.61     5.61     6.72  79.19   

                             relh-1h  ...  sknt  sknt-1h  sknt-2h  sknt-3h  \
station dttime_hour                   ...                                    
AAF     2020-01-01 05:00:00    82.12  ...   0.0      0.0      0.0      3.0   
        2020-01-01 06:00:00    78.66  ...   0.0      0.0      0.0      0.0   
        2020-01-01 07:00:00    81.98  ...   0.0      0.0      0.0      0.0   
        2020-01-01 08:00:00    81.98  ...   0.0      0.0      0.0      0.0   
        2020-01-01 09:00:00    81.98  ...   0.0      0.0      0.0      0.0   

                             sknt-4h  p01i  p01i-1h  p01i-2h  p01i-3h  p01i-4h  
station dttime_hour                                                             
AAF     2020-01-01 05:00:00      3.0   0.0      0.0      0.0      0.0      0.0  
        2020-01-01 06:00:00      3.0   0.0      0.0      0.0      0.0      0.0  
        2020-01-01 07:00:00      0.0   0.0      0.0      0.0      0.0      0.0  
        2020-01-01 08:00:00      0.0   0.0      0.0      0.0      0.0      0.0  
        2020-01-01 09:00:00      0.0   0.0      0.0      0.0      0.0      0.0  

[5 rows x 23 columns]

In [ ]:
dfwstt2.shape

(1260858, 23)

In [ ]:
dict_test = weather_dict2.copy()

In [ ]:
dict_test.keys()

dict_keys(['AAF', 'BOW', 'BCT', 'BKV', 'FIN', 'XMR', 'VSH', 'NFJ', 'CLW', '2IS', 'CEW', 'CTY', 'CGC', 'DAB', '54J', 'DED', 'DTS', 'EGI', 'VPS', 'FHB', 'FXE', 'FMY', 'FLL', 'RSW', 'FPR', 'GNV', 'HWO', 'HST', 'HRT', 'IMM', 'X40', 'INF', 'CRG', 'JAX', 'NIP', 'VQQ', 'NEN', '42J', 'EYW', 'NQX', 'ISM', 'LCQ', 'LAL', 'X07', 'LNA', 'LEE', '24J', 'MCF', 'MTH', 'MKY', 'MAI', 'NRB', 'MLB', 'MIA', 'TMB', 'OPF', 'NDZ', 'APF', 'TTS', 'EVB', 'OCF', 'OBE', 'ORL', 'MCO', 'OMN', '28J', 'XFL', 'ECP', 'PFN', 'TDR', 'COF', 'NPA', 'PNS', 'FPY', '40J', 'PCM', 'PMP', 'PGD', '2J9', 'PIE', 'SPG', 'SFB', 'SRQ', 'SEF', 'SGJ', 'TLH', 'TPA', 'TPF', 'VVG', 'TIX', '1J0', 'BCR', 'PAM', 'X26', 'VDF', 'VNC', 'VRB', 'F45', 'PBI', 'NSE', 'X60', 'GIF', 'SUA', 'ZPH'])

In [ ]:
weather_dict3 = {}
for key, value in weather_dict2.items():
  df = value.copy()
  #group_cols = ['dttime_hour']
  #metric_cols = df.columns

  aggs = df.groupby(level=0).mean()
  #df.drop(metric_cols, axis=1, inplace=True)
  aggs.drop_duplicates(keep='last', inplace=True)

  temp_dict = aggs.to_dict(orient='index')
  weather_dict3[key] = temp_dict

In [ ]:
from timeit import default_timer as timer
def get_wstts_data(ptlight, dtstart, stids):
    cls = ['stid', 'dist',  
    'tmpc', 
    'tmpc-1h', 'tmpc-2h','tmpc-3h', 'tmpc-4h', 
    'relh', 
    'relh-1h', 'relh-2h', 'relh-3h', 'relh-4h', 
    'sknt',
    'sknt-1h', 'sknt-2h', 'sknt-3h', 'sknt-4h', 
    'p01i', 
    'p01i-1h', 'p01i-2h', 'p01i-3h', 'p01i-4h']
    dfwstt = pd.DataFrame(columns=cls)
    for s in stids:
        #t1 = timer()
        
        #df = dfwstt2[(pd.to_datetime(dtstart).round('1h') == pd.to_datetime(dfwstt2.datetime).round('1h')) & (dfwstt2.station == s)]
        #df = dfwstt2.loc[(dfwstt2.index).isin([str(pd.to_datetime(dtstart).round('1h'))], level=1) & dfwstt2.index.isin([s], level=0)]
        #df1 = dfwstt2.loc[dfwstt2.index.isin([s], level=0)]
        #df1 = weather_dict2[s]
        #df1 = weather_dict3[s]
        #t2 = timer()
        #print("1-" + str(t2-t1))
        #df = df1.loc[(df1.index).isin([str(pd.to_datetime(dtstart).round('1h'))])]
        #temp_df = pd.DataFrame([[pd.to_datetime(dtstart).round('1h'), s]], columns=['dttime_hour', 'station'])
        #df = weather_dict3[s].get(pd.to_datetime(dtstart).round('1h'))
        #df = temp_df.merge(dfwstt2)

        switch = 'mixed'

        if switch == 'mixed':
          df1 = weather_dict2[s]
          #t2 = timer()
          #print("1-" + str(t2-t1))
          df = df1.loc[(df1.index).isin([str(pd.to_datetime(dtstart).round('1h'))])]
        elif switch == 'dict':
          df1 = weather_dict3[s]
          #t2 = timer()
          #print("1-" + str(t2-t1))
          df = weather_dict3[s].get(pd.to_datetime(dtstart).round('1h'))

        if not len(df):
            continue
        
        #if not df:
        #  continue

        #print(df.head())
        ptstt = (df.lat.iloc[0], df.lon.iloc[0])
        #ptstt = (df['lat'], df['lon'])
        #t3 = timer()
        #print("2-" + str(t3-t2))
        dist = haversine(ptlight, ptstt)
        #t4 = timer()
        #print("3-" + str(t4-t3))
        dct = {}
        dct['stid'] = s
        dct['dist'] = dist
        knm = ['tmpc', 'relh', 'sknt', 'p01i']
        for k in knm:
            dct[k] = round(df[k].mean(), 2)
            #dct[k] = round(df[k], 2)

        for i in [1, 2, 3, 4]:     
            for k in knm:
                nm = f'{k}-{i}h'
                dct[nm] = round(df[nm].mean(),2)
                #dct[nm] = round(df[nm],2)
        #t5 = timer()
        #print("4-" + str(t5-t4))

        dfwstt = dfwstt.append(pd.DataFrame([dct]))
        #t6 = timer()
        #print("5-" + str(t6-t5))

    return dfwstt

In [ ]:
from timeit import default_timer as timer
def get_wstts_with_range(df, buffrd,fname):
  #t_prev = timer()
  gdflg = gpd.GeoDataFrame(data=df, geometry=gpd.points_from_xy(df.lon, df.lat))
  gdflg["geombuf"] = gdflg.geometry.buffer(buffrd)
  gdflg.set_geometry(col="geombuf", inplace=True, drop=True)
  #t_curr = timer()
  #print("1-" + str(t_curr-t_prev))
  #t_prev = t_curr
  gdfstt = gpd.GeoDataFrame(data=dfstt, geometry=gpd.points_from_xy(dfstt.lon, dfstt.lat))

  dfit2 = pd.DataFrame(columns=['index', 'dttime', 'lat', 'lon',
    'tmpc', 
    'tmpc-1h', 'tmpc-2h','tmpc-3h', 'tmpc-4h', 
    'relh', 
    'relh-1h', 'relh-2h', 'relh-3h', 'relh-4h', 
    'sknt',
    'sknt-1h', 'sknt-2h', 'sknt-3h', 'sknt-4h', 
    'p01i', 
    'p01i-1h', 'p01i-2h', 'p01i-3h', 'p01i-4h'])

  dfmd = pd.DataFrame(columns=['index', 'dttime',
    'tmpc', 
    'tmpc-1h', 'tmpc-2h','tmpc-3h', 'tmpc-4h', 
    'relh', 
    'relh-1h', 'relh-2h', 'relh-3h', 'relh-4h', 
    'sknt',
    'sknt-1h', 'sknt-2h', 'sknt-3h', 'sknt-4h', 
    'p01i', 
    'p01i-1h', 'p01i-2h', 'p01i-3h', 'p01i-4h'])
  #t_curr = timer()
  #print("2-" + str(t_curr-t_prev))
  #t_prev = t_curr
  for i in tqdm(gdflg.index):
      #t_curr = timer()
      #print("3-" + str(t_curr-t_prev))
      #t_prev = t_curr
      it = gpd.sjoin(gdflg[gdflg.index == i], gdfstt, how='inner', predicate='intersects')
      if not len(it):
          continue
      #t_curr = timer()
      #print("4-" + str(t_curr-t_prev))
      #t_prev = t_curr
      pt = (it.lat_left.iloc[0], it.lon_left.iloc[0])
      #dtstart = datetime.strptime(it.dttime.iloc[0][:-3], '%Y-%m-%d %H:%M:%S.%f')
      it['dttime'] = pd.to_datetime(it['dttime'])
      dtstart = it.dttime.iloc[0].to_pydatetime()
      #t_curr = timer()
      #print("5-" + str(t_curr-t_prev))
      #t_prev = t_curr
      df = get_wstts_data(pt, dtstart, list(it.stid))
      #t_curr = timer()
      #print("6-" + str(t_curr-t_prev))
      #t_prev = t_curr
      if not len(df):
          continue

      df = df.dropna()
      df['dist2']=1/df.dist
      df['dist3']=df.dist2/df.dist2.sum()
      #ic(df.dist3)

      knm = ['tmpc', 'relh', 'sknt', 'p01i']
      dct = {}
      dct['idx'] =  i
      dct['dttime'] = dtstart
      for k in knm:

          for i in [1, 2, 3, 4]:
              nm = f'{k}-{i}h'
              dct[nm] = round((df[nm] * df.dist3).sum(), 2)
      #print("7" + str(t1-timer()))
      dfmd = dfmd.append(pd.DataFrame([dct]))
      #print("8" + str(t1-timer()))


  dfmd['index'] = dfmd['idx'].astype(int)
  dfmd.drop(['idx'],axis=1,inplace = True)
  dfmd.set_index('index', inplace = True)

  dfit2 = pd.merge(gdflg.drop(columns=['geometry']), dfmd, left_index=True, right_index=True)
  dfit2.drop(['dttime_y'],axis=1,inplace = True)
  dfit2.rename(columns={'dttime_x': 'dttime'}, inplace=True)

  dfmd.to_csv(f"{pth_output}/stp2_dfmd_{fname}.csv")
  return dfmd

In [ ]:
dflg.head()

,dttime,lat,lon,flash_area,flash_energy,flash_quality_flag,geometry,geombuf
0,2020-09-01 15:56:53.642332077,29.399868,-82.883220,1.252098e+09,3.942836e-13,0.0,POINT (-82.88322 29.39987),"POLYGON ((-82.48322 29.39987, -82.48515 29.360..."
1,2020-09-01 15:54:37.409402847,29.364891,-82.836250,2.083015e+08,1.972844e-13,0.0,POINT (-82.83625 29.36489),"POLYGON ((-82.43625 29.36489, -82.43818 29.325..."
2,2020-09-01 15:21:55.164800643,29.748596,-82.283790,6.973905e+07,1.128511e-14,0.0,POINT (-82.28379 29.74860),"POLYGON ((-81.88379 29.74860, -81.88572 29.709..."
3,2020-09-01 15:21:53.876558303,27.976433,-82.647300,4.765756e+08,7.328486e-14,0.0,POINT (-82.64730 27.97643),"POLYGON ((-82.24730 27.97643, -82.24923 27.937..."
4,2020-09-01 15:10:51.160070419,28.976896,-82.880585,2.765146e+08,1.382846e-13,0.0,POINT (-82.88058 28.97690),"POLYGON ((-82.48058 28.97690, -82.48251 28.937..."


In [ ]:
import multiprocessing as mp
pool = mp.Pool(processes=mp.cpu_count())
mp.cpu_count()

2

In [ ]:
buffrd = 0.4 #40km
dflg = dflg.head(100000)
dfmd = get_wstts_with_range(dflg, buffrd,'04')
dfmd.head()

  0%|          | 0/100000 [00:00<?, ?it/s]

A streamkimeneten csak az utolsó 5000 sor látható.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Discarding nonzero nanoseconds in conversion
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Discarding nonzero nanoseconds in conversion
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Discarding nonzero nanoseconds in conversion
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Discarding nonzero nanoseconds in conversion
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Discarding nonzero nanoseconds in conversion
  This is s

KeyboardInterrupt: ignored

In [ ]:
for k in knm:
            df_temp = df[(df.datetime.dt.date == dtstart.date())]
            dct[k] = round(df_temp[k].mean(), 2)

        for i in [1, 2, 3, 4]:
            dfsmp = df[(df.datetime >= dtstart - timedelta(days=i)) & (df.datetime < dtstart - timedelta(days=i-1)) &(df.datetime <= dtstart)]
     
            for k in knm:
                nm = f'{k}-{i}d'
                dct[nm] = round(dfsmp[k].mean(),2)

In [ ]:
def process_weather_station(df):
  df_new = pd.DataFrame()
  cols = ['tmpc', 'relh', 'sknt', 'p01i']
  for col in cols:
    df_temp = df[(df.datetime.dt.round('1h') == dtstart.date().round('1h'))]
    dct[k] = round(df_temp[k].mean(), 2)

In [ ]:
def get_wstts_data(ptlight, dtstart, stids):
    """
    """
    #cls = ['station', 'datetime', 'lon', 'lat', 
    cls = ['stid', 'dist',  
    'tmpc', 
    'tmpc-1d', 'tmpc-2d','tmpc-3d', 'tmpc-4d', 
    'relh', 
    'relh-1d', 'relh-2d', 'relh-3d', 'relh-4d', 
    'sknt', 
    'sknt-1d', 'sknt-2d', 'sknt-3d', 'sknt-4d', 
    'p01i', 
    'p01i-1d', 'p01i-2d', 'p01i-3d', 'p01i-4d']
    dfwstt = pd.DataFrame(columns=cls)
    for s in stids:
        df = weather_dict[s]
        if not len(df):
            continue
        #df['datetime'] = pd.to_datetime(df['datetime'])
        ptstt = (df.lat.iloc[0], df.lon.iloc[0])
        #print(ptstt, ptlight)
        dist = haversine(ptlight, ptstt)

        dct = {}
        dct['stid'] = s
        dct['dist'] = dist
        knm = ['tmpc', 'relh', 'sknt', 'p01i']
        for k in knm:
            df_temp = df[(df.datetime.dt.date == dtstart.date())]
            dct[k] = round(df_temp[k].mean(), 2)

        for i in [1, 2, 3, 4]:
            dfsmp = df[(df.datetime >= dtstart - timedelta(days=i)) & (df.datetime < dtstart - timedelta(days=i-1)) &(df.datetime <= dtstart)]
     
            for k in knm:
                nm = f'{k}-{i}d'
                dct[nm] = round(dfsmp[k].mean(),2)

        #print(dct)

        dfwstt = dfwstt.append(pd.DataFrame([dct]))

    return dfwstt

In [ ]:
def get_wstts_with_range(df, date, buffrd,fname):
  gdflg = gpd.GeoDataFrame(data=df, geometry=gpd.points_from_xy(df.lon, df.lat))
  gdflg["geombuf"] = gdflg.geometry.buffer(buffrd)
  gdflg.set_geometry(col="geombuf", inplace=True, drop=True)

  gdfstt = gpd.GeoDataFrame(data=dfstt, geometry=gpd.points_from_xy(dfstt.lon, dfstt.lat))

  dfit2 = pd.DataFrame(columns=['index', 'dttime', 'lat', 'lon',
      'tmpc', 
      'tmpc-1d', 'tmpc-2d','tmpc-3d', 'tmpc-4d', 
      'relh', 
      'relh-1d', 'relh-2d', 'relh-3d', 'relh-4d', 
      'sknt', 
      'sknt-1d', 'sknt-2d', 'sknt-3d', 'sknt-4d', 
      'p01i',
      'p01i-1d', 'p01i-2d', 'p01i-3d', 'p01i-4d'])

  dfmd = pd.DataFrame(columns=['index', 'dttime',
      'tmpc', 
      'tmpc-1d', 'tmpc-2d','tmpc-3d', 'tmpc-4d', 
      'relh', 
      'relh-1d', 'relh-2d', 'relh-3d', 'relh-4d', 
      'sknt', 
      'sknt-1d', 'sknt-2d', 'sknt-3d', 'sknt-4d', 
      'p01i',
      'p01i-1d', 'p01i-2d', 'p01i-3d', 'p01i-4d'])
  
  for i in tqdm(gdflg.index):
      it = gpd.sjoin(gdflg[gdflg.index == i], gdfstt, how='inner', predicate='intersects')
      if not len(it):
          continue
      
      pt = (it.lat_left.iloc[0], it.lon_left.iloc[0])
      dtstart = date#.to_pydatetime()
      df = get_wstts_data(pt, dtstart, list(it.stid))
      if not len(df):
          continue

      #ic(df)
      df = df.dropna()
      df['dist2']=1/df.dist
      df['dist3']=df.dist2/df.dist2.sum()
      #ic(df.dist3)

      knm = ['tmpc', 'relh', 'sknt', 'p01i']
      dct = {}
      dct['idx'] =  i
      dct['dttime'] = dtstart
      for k in knm:

          for i in [1, 2, 3, 4]:
              nm = f'{k}-{i}d'
              dct[nm] = round((df[nm] * df.dist3).sum(), 2)

      dfmd = dfmd.append(pd.DataFrame([dct]))

  dfmd['index'] = dfmd['idx'].astype(int)
  dfmd.drop(['idx'],axis=1,inplace = True)
  dfmd.set_index('index', inplace = True)
  print(dfmd.shape)

  dfit2 = pd.merge(gdflg.drop(columns=['geometry']), dfmd, left_index=True, right_index=True)
  dfit2.drop(['dttime_y'],axis=1,inplace = True)
  dfit2.rename(columns={'dttime_x': 'dttime'}, inplace=True)
  print(dfit2.shape)

  dfmd.to_csv(f"{pth_output}/stp2_dfmd_{fname}.csv")
  return dfmd

In [ ]:
import datetime
df_list = []
for lat in tqdm(np.arange(24.5, 31.1, 0.01)):
  for lon in np.arange(-87.7, -80, 0.01):
    df_list.append(
         {
            'lat': round(lat,3),
            'lon': round(lon,3),
        }
    )
df = pd.DataFrame(df_list)
df.head()

  0%|          | 0/661 [00:00<?, ?it/s]

,lat,lon
0,24.5,-87.70
1,24.5,-87.69
2,24.5,-87.68
3,24.5,-87.67
4,24.5,-87.66


In [ ]:
date = datetime.datetime(2020, 5, 17)    
dfmd = get_wstts_with_range(df, date, 0.4,'04')

  0%|          | 0/509631 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

KeyboardInterrupt: ignored

In [ ]:
509631 * 365

186015315

# Step 2 : WSTTS

In [ ]:
dfit = pd.read_csv(f"{pth_output}/stp21_wstt.csv",index_col=0)
dfit['dttime'] = pd.to_datetime(dfit.dttime).dt.round('ms')
print(dfit.shape)
dfit.head()

(162, 22)


,index,dttime,lat,lon,flash_area,flash_energy,flash_quality_flag,FID,Label,landcover,...,ERC-2d,ERC-3d,RH_Min,RH_Min-1d,RH_Min-2d,RH_Min-3d,RH_Max,RH_Max-1d,RH_Max-2d,RH_Max-3d
3,94761,2020-07-21 18:48:32.896,28.902424,-82.295650,3.447276e+08,1.722845e-13,0.0,1598,Fire,2,...,22.71,21.49,50.69,50.69,50.69,59.84,95.77,95.77,97.53,97.53
4,94771,2020-07-21 18:51:40.680,28.905050,-82.306786,2.069281e+08,2.828504e-14,0.0,206,Fire,2,...,22.81,21.60,50.57,50.57,50.57,59.68,95.74,95.74,97.47,97.47
5,94828,2020-07-21 18:52:29.924,28.906400,-82.308820,2.069281e+08,2.828504e-14,0.0,206,Fire,2,...,22.82,21.61,50.56,50.56,50.56,59.66,95.73,95.73,97.46,97.46
6,99962,2020-07-01 16:46:58.648,28.959904,-82.381830,1.450176e+09,8.122819e-13,0.0,1034,Fire,2,...,26.00,26.00,58.00,58.00,54.00,49.00,96.00,96.00,96.00,96.00
7,104996,2020-04-20 15:52:06.956,28.810152,-81.091210,1.373417e+08,4.328498e-14,0.0,262,Fire,2,...,20.77,14.71,44.74,44.74,62.57,67.04,94.50,94.50,97.76,98.63


In [ ]:
dfit = pd.read_csv(f"{pth_output}/stp1_landcover.csv",index_col=0)
dfit['dttime'] = pd.to_datetime(dfit.dttime).dt.round('ms')
print(dfit.shape)
dfit.head()

(5978004, 8)


,index,dttime,lat,lon,flash_area,flash_energy,flash_quality_flag,landcover
0,0,2020-09-01 15:56:53.642,29.399868,-82.883220,1.252098e+09,3.942836e-13,0.0,2.0
1,1,2020-09-01 15:54:37.409,29.364891,-82.836250,2.083015e+08,1.972844e-13,0.0,2.0
2,2,2020-09-01 15:21:55.165,29.748596,-82.283790,6.973905e+07,1.128511e-14,0.0,2.0
3,3,2020-09-01 15:21:53.877,27.976433,-82.647300,4.765756e+08,7.328486e-14,0.0,1.0
4,4,2020-09-01 15:10:51.160,28.976896,-82.880585,2.765146e+08,1.382846e-13,0.0,1.0


In [ ]:
#CHANGE TO YOUR FILE LOCATION
wsttdata = "/content/drive/MyDrive/FNN/dangerous_lightnings/1_data_download/weather_station.csv"
dfstt = pd.read_csv(wsttdata)

In [ ]:
weather_dict = {}
for stid in tqdm(dfstt.stid):
  #CHANGE TO YOUR FILE LOCATION
  wstt_path = f'/content/drive/MyDrive/FNN/dangerous_lightnings/1_data_download/output/weather/{stid}.csv'
  data = pd.read_csv(wstt_path,index_col=0)
  data[['tmpc','relh', 'sknt', 'p01i']] = data[['tmpc','relh', 'sknt', 'p01i']].apply(pd.to_numeric, errors='coerce')
  data = data.dropna()
  data['datetime'] = pd.to_datetime(data['valid'])
  data = data.drop('valid', axis=1)
  data = data.set_index('datetime').groupby(pd.Grouper(freq='D')).mean()
  weather_dict[stid]=data

  0%|          | 0/104 [00:00<?, ?it/s]

In [ ]:
weather_dict['2IS'].head()

,lon,lat,tmpc,relh,sknt,p01i
datetime,,,,,,
2020-09-22,-81.0468,26.7339,26.254286,70.831429,10.000000,0.025714
2020-09-23,-81.0468,26.7339,24.941970,72.795606,6.242424,0.000000
2020-09-24,-81.0468,26.7339,25.585915,84.102958,7.676056,0.000000
2020-09-25,-81.0468,26.7339,26.602206,87.254559,4.573529,0.017941
2020-09-26,-81.0468,26.7339,26.585278,86.487083,3.958333,0.000000


In [ ]:
def get_wstts_data(ptlight, dtstart, stids):
    """
    """
    #cls = ['station', 'datetime', 'lon', 'lat', 
    cls = ['stid', 'dist',  
    'tmpc', 'tmpc-1h', 'tmpc-2h','tmpc-3h', 'tmpc-4h', 
    'tmpc-1d', 'tmpc-2d','tmpc-3d', 'tmpc-4d', 
    'relh', 'relh-1h', 'relh-2h', 'relh-3h', 'relh-4h', 
    'relh-1d', 'relh-2d', 'relh-3d', 'relh-4d', 
    'sknt', 'sknt-1h', 'sknt-2h', 'sknt-3h', 'sknt-4h', 
    'sknt-1d', 'sknt-2d', 'sknt-3d', 'sknt-4d', 
    'p01i', 'p01i-1h', 'p01i-2h', 'p01i-3h', 'p01i-4h',
    'p01i-1d', 'p01i-2d', 'p01i-3d', 'p01i-4d']
    dfwstt = pd.DataFrame(columns=cls)
    for s in stids:
        df = weather_dict[s]
        if not len(df):
            continue
        #df['datetime'] = pd.to_datetime(df['datetime'])
        ptstt = (df.lat.iloc[0], df.lon.iloc[0])
        #print(ptstt, ptlight)
        dist = haversine(ptlight, ptstt)

        dct = {}
        dct['stid'] = s
        dct['dist'] = dist
        knm = ['tmpc', 'relh', 'sknt', 'p01i']
        for k in knm:
            df_temp = df[(df.datetime.dt.date == dtstart.date())]
            dct[k] = round(df_temp[k].mean(), 2)

        for i in [1, 2, 3, 4]:
            dfsmp = df[(df.datetime >= dtstart - timedelta(hours=i)) & (df.datetime <= dtstart)]
     
            for k in knm:
                nm = f'{k}-{i}h'
                dct[nm] = round(dfsmp[k].mean(),2)

        for i in [1, 2, 3, 4]:
            dfsmp = df[(df.datetime >= dtstart - timedelta(days=i)) & (df.datetime < dtstart - timedelta(days=i-1)) &(df.datetime <= dtstart)]
     
            for k in knm:
                nm = f'{k}-{i}d'
                dct[nm] = round(dfsmp[k].mean(),2)

        #print(dct)

        dfwstt = dfwstt.append(pd.DataFrame([dct]))

    return dfwstt

In [ ]:
def get_wstts_with_range(buffrd,fname):
  gdflg = gpd.GeoDataFrame(data=dfit, geometry=gpd.points_from_xy(dfit.lon, dfit.lat))
  gdflg["geombuf"] = gdflg.geometry.buffer(buffrd)
  gdflg.set_geometry(col="geombuf", inplace=True, drop=True)

  gdfstt = gpd.GeoDataFrame(data=dfstt, geometry=gpd.points_from_xy(dfstt.lon, dfstt.lat))

  dfit2 = pd.DataFrame(columns=['index', 'dttime', 'lat', 'lon', 'polarity', 'cloud_ground', 
      'ell_smajor', 'ell_sminor', 'ell_angle', 'qsquared', 
      'tmpc', 'tmpc-1h', 'tmpc-2h','tmpc-3h', 'tmpc-4h', 
      'tmpc-1d', 'tmpc-2d','tmpc-3d', 'tmpc-4d', 
      'relh', 'relh-1h', 'relh-2h', 'relh-3h', 'relh-4h', 
      'relh-1d', 'relh-2d', 'relh-3d', 'relh-4d', 
      'sknt', 'sknt-1h', 'sknt-2h', 'sknt-3h', 'sknt-4h', 
      'sknt-1d', 'sknt-2d', 'sknt-3d', 'sknt-4d', 
      'p01i', 'p01i-1h', 'p01i-2h', 'p01i-3h', 'p01i-4h',
      'p01i-1d', 'p01i-2d', 'p01i-3d', 'p01i-4d'])

  dfmd = pd.DataFrame(columns=['index', 'dttime',
      'tmpc', 'tmpc-1h', 'tmpc-2h','tmpc-3h', 'tmpc-4h', 
      'tmpc-1d', 'tmpc-2d','tmpc-3d', 'tmpc-4d', 
      'relh', 'relh-1h', 'relh-2h', 'relh-3h', 'relh-4h', 
      'relh-1d', 'relh-2d', 'relh-3d', 'relh-4d', 
      'sknt', 'sknt-1h', 'sknt-2h', 'sknt-3h', 'sknt-4h', 
      'sknt-1d', 'sknt-2d', 'sknt-3d', 'sknt-4d', 
      'p01i', 'p01i-1h', 'p01i-2h', 'p01i-3h', 'p01i-4h',
      'p01i-1d', 'p01i-2d', 'p01i-3d', 'p01i-4d'])
  
  for i in tqdm(gdflg.index):
      it = gpd.sjoin(gdflg[gdflg.index == i], gdfstt, how='inner', predicate='intersects')
      if not len(it):
          continue
      
      pt = (it.lat_left.iloc[0], it.lon_left.iloc[0])
      dtstart = it.dttime.iloc[0].to_pydatetime()
      df = get_wstts_data(pt, dtstart, list(it.stid))
      if not len(df):
          continue

      #ic(df)
      df = df.dropna()
      df['dist2']=1/df.dist
      df['dist3']=df.dist2/df.dist2.sum()
      #ic(df.dist3)

      knm = ['tmpc', 'relh', 'sknt', 'p01i']
      dct = {}
      dct['idx'] =  i
      dct['dttime'] = dtstart
      for k in knm:
          dct[k] = round((df[k] * df.dist3).sum(), 2)
          for i in [1, 2, 3, 4]:
              nm = f'{k}-{i}h'
              dct[nm] = round((df[nm] * df.dist3).sum(), 2)

          for i in [1, 2, 3, 4]:
              nm = f'{k}-{i}d'
              dct[nm] = round((df[nm] * df.dist3).sum(), 2)

      dfmd = dfmd.append(pd.DataFrame([dct]))

  dfmd['index'] = dfmd['idx'].astype(int)
  dfmd.drop(['idx'],axis=1,inplace = True)
  dfmd.set_index('index', inplace = True)
  print(dfmd.shape)

  dfit2 = pd.merge(gdflg.drop(columns=['geometry']), dfmd, left_index=True, right_index=True)
  dfit2.drop(['dttime_y'],axis=1,inplace = True)
  dfit2.rename(columns={'dttime_x': 'dttime'}, inplace=True)
  print(dfit2.shape)

  dfmd.to_csv(f"{pth_output}/stp2_dfmd_{fname}.csv")
  return dfmd

In [ ]:
buffrd = 0.4 #40km
dfmd = get_wstts_with_range(buffrd,'04')
dfmd.head()

  0%|          | 0/5746 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

(5632, 37)
(5632, 49)


,dttime,tmpc,tmpc-1h,tmpc-2h,tmpc-3h,tmpc-4h,tmpc-1d,tmpc-2d,tmpc-3d,tmpc-4d,...,sknt-4d,p01i,p01i-1h,p01i-2h,p01i-3h,p01i-4h,p01i-1d,p01i-2d,p01i-3d,p01i-4d
index,,,,,,,,,,,,,,,,,,,,,
1,2020-07-21 18:48:32.896,27.39,31.69,32.14,32.14,31.88,28.67,28.21,27.11,26.97,...,3.05,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00
2,2020-07-01 16:46:58.648,26.56,28.63,30.15,30.21,29.76,28.66,28.91,28.67,27.70,...,1.72,0.01,0.02,0.01,0.01,0.00,0.00,0.00,0.00,0.00
3,2020-04-20 15:52:06.956,26.82,24.84,25.19,25.39,25.44,27.89,24.50,24.00,20.06,...,9.70,0.02,0.05,0.04,0.03,0.03,0.01,0.02,0.01,0.00
6,2020-07-14 23:09:58.038,30.47,31.82,32.82,33.04,33.11,30.52,29.59,28.59,28.05,...,6.98,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03
7,2020-07-14 23:01:22.320,30.47,32.33,32.83,33.12,33.12,30.53,29.58,28.53,28.07,...,6.95,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03


In [ ]:
dfmd = pd.read_csv(f"{pth_output}/stp2_dfmd_04.csv",index_col=0)

dfit['dttime'] = dfit['dttime'].dt.to_pydatetime()
print(dfit.shape)
print(dfmd.shape)
dfit = dfit.loc[~dfit['dttime'].isin(dfmd.dttime.unique())]
print(dfit.shape)

buffrd = 1 #40km
dfmd = get_wstts_with_range(buffrd,'1')
dfmd.head()

(5746, 15)
(5632, 37)
(114, 15)


  0%|          | 0/114 [00:00<?, ?it/s]

(114, 37)
(114, 49)


,dttime,tmpc,tmpc-1h,tmpc-2h,tmpc-3h,tmpc-4h,tmpc-1d,tmpc-2d,tmpc-3d,tmpc-4d,...,sknt-4d,p01i,p01i-1h,p01i-2h,p01i-3h,p01i-4h,p01i-1d,p01i-2d,p01i-3d,p01i-4d
index,,,,,,,,,,,,,,,,,,,,,
78,2020-06-27 18:58:30.328,27.81,31.79,31.85,31.94,31.81,28.93,28.18,27.35,27.36,...,4.64,0.0,0.00,0.01,0.01,0.01,0.0,0.0,0.0,0.0
79,2020-06-27 18:50:06.793,27.83,31.97,31.99,32.03,31.75,29.00,28.14,27.37,27.45,...,4.65,0.0,0.02,0.01,0.01,0.00,0.0,0.0,0.0,0.0
81,2020-06-27 18:55:52.529,27.83,31.56,31.72,31.89,31.78,28.91,28.16,27.38,27.47,...,4.55,0.0,0.00,0.01,0.01,0.01,0.0,0.0,0.0,0.0
82,2020-06-27 18:52:54.503,27.83,31.84,31.87,32.01,31.79,28.99,28.14,27.37,27.45,...,4.64,0.0,0.02,0.01,0.01,0.00,0.0,0.0,0.0,0.0
87,2020-06-27 18:51:08.851,27.83,31.86,31.88,32.01,31.80,28.99,28.14,27.37,27.45,...,4.64,0.0,0.02,0.01,0.01,0.00,0.0,0.0,0.0,0.0


In [ ]:
dfmd1 = pd.read_csv(f"{pth_output}/stp2_dfmd_04.csv",index_col=0)
dfmd2 = pd.read_csv(f"{pth_output}/stp2_dfmd_1.csv",index_col=0)
dfmd = dfmd1.append(dfmd2)
dfmd.shape

(5746, 37)

In [ ]:
dfit = pd.read_csv(f"{pth_output}/stp1_landcover.csv",index_col=0)
dfit['dttime'] = pd.to_datetime(dfit.dttime).dt.round('ms')
buffrd = 1 #40km
gdflg = gpd.GeoDataFrame(data=dfit, geometry=gpd.points_from_xy(dfit.lon, dfit.lat))
gdflg["geombuf"] = gdflg.geometry.buffer(buffrd)
gdflg.set_geometry(col="geombuf", inplace=True, drop=True)

In [ ]:
dfit2 = pd.merge(gdflg.drop(columns=['geometry']), dfmd, left_index=True, right_index=True)
dfit2.drop(['dttime_y'],axis=1,inplace = True)
dfit2.rename(columns={'dttime_x': 'dttime'}, inplace=True)
print(dfit2.shape)
dfit2.head()

(5746, 49)


,index,dttime,lat,lon,flash_area,flash_energy,flash_quality_flag,FID,Label,dttime_round,...,sknt-4d,p01i,p01i-1h,p01i-2h,p01i-3h,p01i-4h,p01i-1d,p01i-2d,p01i-3d,p01i-4d
1,94761,2020-07-21 18:48:32.896,28.902424,-82.295650,3.447276e+08,1.722845e-13,0.0,1598.0,Fire,NaN,...,3.05,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00
2,99962,2020-07-01 16:46:58.648,28.959904,-82.381830,1.450176e+09,8.122819e-13,0.0,1034.0,Fire,NaN,...,1.72,0.01,0.02,0.01,0.01,0.00,0.00,0.00,0.00,0.00
3,104996,2020-04-20 15:52:06.956,28.810152,-81.091210,1.373417e+08,4.328498e-14,0.0,262.0,Fire,NaN,...,9.70,0.02,0.05,0.04,0.03,0.03,0.01,0.02,0.01,0.00
6,123755,2020-07-14 23:09:58.038,28.491884,-81.183280,1.367313e+08,3.028503e-14,0.0,746.0,Fire,NaN,...,6.98,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03
7,123757,2020-07-14 23:01:22.320,28.492050,-81.186455,1.367313e+08,2.828504e-14,0.0,746.0,Fire,NaN,...,6.95,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03


In [ ]:
dfit2.to_csv(f"{pth_output}/stp2_wstt.csv")

# Step 3 : Precipitation

In [ ]:
dfit = pd.read_csv(f"{pth_output}/stp2_wstt.csv",index_col=0)
dfit['dttime'] = pd.to_datetime(dfit.dttime).dt.round('ms')
print(dfit.shape)
dfit.head()

(52675, 49)


,index,dttime,lat,lon,flash_area,flash_energy,flash_quality_flag,FID,Label,dttime_round,...,sknt-4d,p01i,p01i-1h,p01i-2h,p01i-3h,p01i-4h,p01i-1d,p01i-2d,p01i-3d,p01i-4d
1,94761,2020-07-21 18:48:32.896,28.902424,-82.295650,3.447276e+08,1.722845e-13,0.0,1598.0,Fire,NaN,...,3.05,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00
2,99962,2020-07-01 16:46:58.648,28.959904,-82.381830,1.450176e+09,8.122819e-13,0.0,1034.0,Fire,NaN,...,1.72,0.01,0.02,0.01,0.01,0.00,0.00,0.00,0.00,0.00
3,104996,2020-04-20 15:52:06.956,28.810152,-81.091210,1.373417e+08,4.328498e-14,0.0,262.0,Fire,NaN,...,9.70,0.02,0.05,0.04,0.03,0.03,0.01,0.02,0.01,0.00
6,123755,2020-07-14 23:09:58.038,28.491884,-81.183280,1.367313e+08,3.028503e-14,0.0,746.0,Fire,NaN,...,6.98,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03
7,123757,2020-07-14 23:01:22.320,28.492050,-81.186455,1.367313e+08,2.828504e-14,0.0,746.0,Fire,NaN,...,6.95,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03


In [ ]:
#dfit = dfit.head(1)

In [ ]:
#CHANGE TO YOUR FILE LOCATION
pth_weather = '/content/drive/MyDrive/FNN/dangerous_lightnings/1_data_download/output/precipitation/processed_tif'
pfx = 'nws_precip_1day_'

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

precip_dict = {}

start_date = dt(2019, 12, 10)
end_date = dt(2021, 1, 10)
total_days = (end_date - start_date).days

for single_date in tqdm(daterange(start_date, end_date),total=total_days):
    fnm = f"{pfx}{single_date.strftime('%Y%m%d')}_conus.tif"
    fp = f"{pth_weather}/{pfx}{single_date.strftime('%Y%m%d')}_conus.tif"
    precip_dict[fnm]=rioxarray.open_rasterio(fp) 

  0%|          | 0/397 [00:00<?, ?it/s]

In [ ]:
def get_pixel_value(date, lat, lon):
    """
    """
    fnm = f"{pfx}{date.strftime('%Y%m%d')}_conus.tif"
    xarr = precip_dict[fnm]

    # Slice one of the bands
    img = xarr[0, :, :] 
    #Use the .sel() method to retrieve the value of the nearest cell close to your POI
    val = img.sel(x=lon, y=lat, method="nearest")    
    #print("This is val: ", val)
    pxbd1=round(float(val), 2)

    img = xarr[3, :, :] 
    #Use the .sel() method to retrieve the value of the nearest cell close to your POI
    val = img.sel(x=lon, y=lat, method="nearest")    
    #print("This is val: ", val)
    pxbd4=round(float(val), 2)

    return pxbd1, pxbd4


dfmd = pd.DataFrame(columns=['index','prec_obs', 'prec_pnormal',
                                'prec_obs-1d', 'prec_pnormal-1d', 
                                'prec_obs-2d', 'prec_pnormal-2d',
                                'prec_obs-sum7d', 'prec_pnormal-avg7d',
                                'prec_obs-sum14d', 'prec_pnormal-avg14d',
                                'prec_obs+1d', 'prec_pnormal+1d',
                                'prec_obs+2d', 'prec_pnormal+2d'])

for index, row in tqdm(dfit.iterrows(), total=dfit.shape[0]):
    date = row.dttime.date()
    lattd = row.lat
    lontd = row.lon

    dct = {}
    dct['index'] = index
    dct['prec_obs'], dct['prec_pnormal'] = get_pixel_value(date, lattd, lontd)
    dct['prec_obs-1d'], dct['prec_pnormal-1d'] = get_pixel_value(date - timedelta(days=1), lattd, lontd)
    dct['prec_obs-2d'], dct['prec_pnormal-2d'] = get_pixel_value(date - timedelta(days=2), lattd, lontd)

    pnml = dct['prec_pnormal-1d'] + dct['prec_pnormal-2d']
    pobs = dct['prec_obs-1d'] + dct['prec_obs-2d']

    for i in range(3,8):
            obs, nml = get_pixel_value(date - timedelta(days=i), lattd, lontd)
            pnml = pnml + nml
            pobs = pobs + obs

    dct['prec_obs-sum7d'] = round(pobs, 2)
    dct['prec_pnormal-avg7d'] = round(pnml/7, 2)

    for i in range(8,15):
        obs, nml = get_pixel_value(date - timedelta(days=i), lattd, lontd)
        pnml = pnml + nml
        pobs = pobs + obs

    dct['prec_obs-sum14d'] = round(pobs, 2)
    dct['prec_pnormal-avg14d'] = round(pnml/14, 2)

    dct['prec_obs+1d'], dct['prec_pnormal+1d'] = get_pixel_value(date + timedelta(days=1), lattd, lontd)
    dct['prec_obs+2d'], dct['prec_pnormal+2d'] = get_pixel_value(date + timedelta(days=2), lattd, lontd)
    dfmd = dfmd.append(pd.DataFrame([dct]))

dfmd.head()

  0%|          | 0/52675 [00:00<?, ?it/s]

,index,prec_obs,prec_pnormal,prec_obs-1d,prec_pnormal-1d,prec_obs-2d,prec_pnormal-2d,prec_obs-sum7d,prec_pnormal-avg7d,prec_obs-sum14d,prec_pnormal-avg14d,prec_obs+1d,prec_pnormal+1d,prec_obs+2d,prec_pnormal+2d
0,1,0.07,31.21,0.00,0.00,0.02,9.86,1.58,94.10,4.97,148.07,0.24,98.56,0.19,80.49
0,2,0.00,0.00,0.00,0.00,0.00,0.00,0.69,37.48,1.04,28.16,0.83,344.88,0.15,62.11
0,3,0.00,0.00,0.31,382.73,0.02,29.44,0.67,118.46,1.30,115.31,1.52,1889.11,0.00,0.00
0,6,0.00,0.00,0.00,0.00,0.28,122.07,2.14,135.27,4.25,134.52,1.06,470.83,0.10,43.60
0,7,0.00,0.00,0.00,0.00,0.27,118.33,1.81,114.11,3.84,121.07,0.61,271.47,0.30,130.52


In [ ]:
print(dfmd.shape)
#dfmd['index'] = dfmd['idx'].astype(int)
#dfmd.drop(['idx'],axis=1,inplace = True)
dfmd.set_index('index', inplace = True)
dfmd.head()

(52675, 15)


,prec_obs,prec_pnormal,prec_obs-1d,prec_pnormal-1d,prec_obs-2d,prec_pnormal-2d,prec_obs-sum7d,prec_pnormal-avg7d,prec_obs-sum14d,prec_pnormal-avg14d,prec_obs+1d,prec_pnormal+1d,prec_obs+2d,prec_pnormal+2d
index,,,,,,,,,,,,,,
1,0.07,31.21,0.00,0.00,0.02,9.86,1.58,94.10,4.97,148.07,0.24,98.56,0.19,80.49
2,0.00,0.00,0.00,0.00,0.00,0.00,0.69,37.48,1.04,28.16,0.83,344.88,0.15,62.11
3,0.00,0.00,0.31,382.73,0.02,29.44,0.67,118.46,1.30,115.31,1.52,1889.11,0.00,0.00
6,0.00,0.00,0.00,0.00,0.28,122.07,2.14,135.27,4.25,134.52,1.06,470.83,0.10,43.60
7,0.00,0.00,0.00,0.00,0.27,118.33,1.81,114.11,3.84,121.07,0.61,271.47,0.30,130.52


In [ ]:
dfit3 = pd.merge(dfit, dfmd, left_index=True, right_index=True)
dfit3.shape

(52675, 63)

In [ ]:
dfit3.to_csv(f"{pth_output}/stp3_precip.csv")

In [ ]:
dfit3.to_csv(f"/content/drive/MyDrive/FNN/dangerous_lightnings/4_match_lightning_with_data/output/{dataset_name}.csv")